In [1]:
import os
import time

import numpy as np
import pandas as pd
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster

import sys
sys.path.append("../python/property")

from address import Address

In [2]:
# cluster = LocalCluster(ip="0.0.0.0")
# client = Client(cluster)
# print(client)
# N_JOBS = 32

In [14]:
addressProperty = Address("address")
address_score = addressProperty.get_address_map_score()
travel_score = addressProperty.get_travel_map_score()

In [5]:
df = pd.read_csv("/home/phongdk/data_user_income_targeting/2019-05-13/location_from_2019-04-30_to_2019-05-13.csv.gz", nrows=None)
df.head()

,user_id,location
0,-9223367349307875591,1025001000
1,-9223363257652853957,1048008000
2,-9223361265171709887,1016001000
3,-9223360657040448554,1025009000
4,-9223360657040448554,1025009000


In [6]:
len(df)

19492607

In [7]:
def get_code2address(filename="external_data/location/Vietnam regions.xlsx"):
    CITY = ["hà nội", "hồ chí minh", "đà nẵng", "hải phòng", "cần thơ"]
    df = pd.read_excel(filename, sheet_name='Districts')
    df.columns = ["district_id", "district_name", "province_id", "province_name"]
    df.set_index("district_id", inplace=True)
    df["district_name"] = df["district_name"].apply(lambda x: x.lower().strip())
    df["province_name"] = df["province_name"].apply(lambda x: x.lower().strip())
    df["address"] = df.apply(lambda x: x["province_name"] if x["province_name"] in CITY else x["district_name"], axis=1)
    new_dict = df["address"].to_dict()
    return new_dict

In [8]:
code2address = get_code2address(filename="../../external_data/location/Vietnam regions.xlsx")

In [9]:
df["province"] = df["location"].apply(lambda x: str(x)[1:4])

In [10]:
df = df.drop_duplicates(subset=["user_id", "province"], keep="first")
df.shape

(8923014, 3)

In [11]:
df["address"] = df["location"].map(code2address).map(address_score).fillna(address_score["others"])

In [12]:
score = df.groupby('user_id').agg({'address' : 'mean', 'province' : 'size'})
score.columns = ['address_score', 'num_travel']

In [13]:
score.head()

,address_score,num_travel
user_id,,
-9223367349307875591,5.0,1
-9223363257652853957,3.5,1
-9223361265171709887,1.0,1
-9223360657040448554,5.0,1
-9223359671335127889,3.5,1


In [20]:
score['travel_score'] = score['num_travel'].map(travel_score).fillna(travel_score["others"])

In [21]:
score[score["travel_score"] > 0]

,address_score,num_travel,travel_score
user_id,,,
-9223290702945974120,4.250000,2,4.0
-9223287828018999744,5.000000,2,4.0
-9223278515497223307,1.000000,2,4.0
-9223273887140903393,1.000000,2,4.0
-9223257631703703369,3.000000,2,4.0
-9223242889001368719,3.500000,2,4.0
-9223236556411434475,2.250000,2,4.0
-9223231434531866120,4.250000,2,4.0
-9223229552374853412,4.250000,2,4.0
